集成学习是目前机器学习的一大热门方向，所谓集成学习简单理解就是指采用多个分类器（回归器）对数据集进行预测，从而提高整体分类器的泛化能力。集成学习的思路大体上可以这样理解：在对新的数据实例进行分类的时候，通过训练好多个分类器，把这些分类器的的分类结果进行某种组合（比如投票）决定分类结果，以取得更好的结果，就是我们生活中那句话“三个臭皮匠顶个诸葛亮”，通过使用多个决策者共同决策一个实例的分类从而提高分类器的泛化能力【引用自csdn[博客](http://www.cnblogs.com/GuoJiaSheng/p/4033584.html)】。
![ensemble](pic/ensemble.png)

  当然通过这种集成学习提高分类器（回归器）的整体泛化能力是有条件的：

- 分类器（回归器）之间应该具有差异性，如果使用的是同一个分类器，那么集成起来的分类结果是不会有变化的。

- 分类器的精度，每个个体分类器的分类精度必须大于0.5。比如分类时，半数以上的分类器判断正确最终结果才能分类正确。

需要解决的两个问题：
- 怎样构建差异性的分类器
- 怎样整合最终的结果

# 1.结果的整合

- 对于回归模型，可以对最终的结果进行简单的平均或者加权平均。

 - 简单平均：$$f = \sum_{i=1}^{N} f_i(x)/N$$
 - 加权平均：$$f = \sum_{i=1}^{N} w_if_i(x)/N$$
- 对于分类模型，可以使用majority vote办法，少数服从多数。比如5个分类器中有三个将样本分为A类，另外两个分别是B、C类，那么最终分类结果就是A。

# 2.构建差异性的分类器（回归器）

这里主要讨论分类器。

## 2.1.通过处理数据集生成差异性分类器

### 2.1.1Bootstrap Aggregation(Bagging)

通过对原数据集进行有放回的采样（bootstrap），构建出大小和原数据集D一样的新数据集$D_1，D_2，D_3.....D_n$，然后用这些新的数据集训练多个分类器$f_1，f_2，f_3....f_n$。因为是有放回的采样，所以一些样本可能会出现多次。

$$f = \sum_{i=1}^{N} f_i(x)/N$$
 

bagging的思想是通过构建多个分类器，计算均值来降低最终结果方差（在统计学上可以理解为求一系列观测值得均值来降低方差）。通过降低基分类器方差改善了泛化能力，因此bagging的性能依赖于基分类器的稳定性，如果基分类器是不稳定的，bagging有助于减低训练数据的随机扰动导致的误差，但是如果基分类器是稳定的，即对数据变化不敏感，那么bagging方法就得不到性能的提升，甚至会减低。

如果bagging的基分类器是决策树，假设数据特征量m，每次训练时构建一棵最大特征数相同的树，假设为p，$p \leq m$，这$p$个特征是决策树依据生成规则自动选取。一般情况下，多个决策树分类器能够降低方差增强泛化能力，但是若数据中存在比较强势的特征，每次生成决策树时都会选取强势特征，势必造成决策树比较类似、关联性比较强，bagging性能不能得到提升。

### 2.1.2Random Forest

随机森林在字面上理解就是随机生成许多决策树。
针对上面bagging中存在的问题，随机森林采取的策略是每次生成决策树时，随机选取p个特征值指定作为决策树的特征，这样强势特征和非强势的特征都有相同的概率被选中，从而生成的决策树之间关联性降低，最终达到降低方差的目的。

随机森林重在构造一系列关联性低的决策树。


### 2.1.3Boosting

提升方法是一个迭代的过程，通过改变样本分布，使得分类器聚集在那些很难分的样本上，对那些容易错分的数据加强学习，增加错分数据的权重，这样错分的数据再下一轮的迭代就有更大的作用（对错分数据进行惩罚）。数据的权重有两个作用，一方面我们可以使用这些权值作为抽样分布，进行对数据的抽样，另一方面分类器可以使用权值学习有利于高权重样本的分类器。

#### 2.1.3.1Boosting与Bagging比较
Bagging采用均匀取样，而Boosting根据错误率来取样，因此Boosting的分类精度要优于Bagging。Bagging的训练集的选择是随机的，各轮训练集之间相互独立，而Boostlng的各轮训练集的选择与前面各轮的学习结果有关；Bagging的各个预测函数没有权重，而Boosting是有权重的；Bagging的各个预测函数可以并行生成，而Boosting的各个预测函数只能顺序生成。对于像神经网络这样极为耗时的学习方法。Bagging可通过并行训练节省大量时间开销。

我们首先通过Adaboost了解提升方法的思路，每轮训练通过改变数据样本权重和基分类的权重来进行优化。

#### 2.1.3.2Adaboost
常用的有Adaboost(adaptive boosting)模型，其算法流程如下（引用于《统计机器学习》）：
- 输入：训练输入为数据集X为$x_1, x_2, ...,x_N$（共有N个样本），其中$x_i \in R^n$(n维向量）。输出标签为$y_1, y_2, ..., y_N$，取值范围为{-1, +1}。
- 输出最终分类器G(x)。


- 步骤：
 - 初始化训练数据的权值分布。
 $$ D_1 = (w_{11}, w_{12}, ..., w_{1N}), \ w_{1i} = \frac{1}{N}, \  其中i=1, 2, ..., N$$
 - 对每次迭代， m=1, 2, ..., M（共有M个生成判别函数）
  - 使用权值$D_m$的训练数据进行学习，得到基本分类器$G_m(x)$
  - 计算当前分类器$G_m(x)$在训练数据集上的误差率
  $$ e_m = P(G_m(x)\neq y_i) = \sum_{i=1}^{N} w_{mi}I(G_m(x_i) \neq y_i)$$
  - 计算当前分类器$G_m(x)$的系数，可以看出当分类误差率越小则系数越大
  $$\alpha_m = \frac{1}{2}log \frac{1-e_m}{e_m}$$
  - 更新训练数据集的权值分布，可以看出分类错误的样本权重变大
  $$ D_{m+1} = (w_{m+1,1}, w_{m+1,2}, ..., w_{m+1,N})$$
  
  $$ w_{m+1,i} = \frac {w_{mi}}{Z_m}exp(-\alpha_my_iG_m(x_i)), i=1, 2, ..., N$$
  
  $$ w_{m+1,i} = \begin{cases}
  \frac {w_{mi}}{Z_m}exp(-\alpha_m), & G_m(x_i) = y_i \\
  \frac {w_{mi}}{Z_m}exp(\alpha_m), & G_m(x_i)  \neq y_i
  \end{cases}$$
  
  其中$Z_m$是归一化因子
  $$Z_m = \sum_{i=1}^{N} w_{mi} exp(-\alpha_m y_iG_m(x_i))$$

 - 构建基本分类器的线性组合
 $$f(x) = \sum_{m=1}^{M}\alpha_mG_m(x)$$
 最终的分类器
 $$G(x) = sign(f(x))=sign(\sum_{m=1}^{M}\alpha_mG_m(x))$$
 
 这里大家可能会问？这只是一个算法计算流程，提升方法模型是什么？损失函数怎么表示？

#### 2.1.3.2提升方法前向分布算法表达

提升方法模型可以表达成加法模型：
$$f(x) = \sum_{m=1}^{M}\beta_mb(x;\gamma_m)$$

其中，$b(x;\gamma_m)$为基函数，可以是决策树，也可以是感知器，$\gamma_m$为基函数的参数，如在Adaboost里是样本的权重，$\beta_m$为分类器系数，如上面Adaboost里面的$\alpha$。

给定训练数据和损失函数条件下，学习加法模型也就是求损失函数最小：

$$min_{\beta_m, \gamma_m} \ \sum_{i=1}^{N}L(y_i, f(x_i))$$

也就是：

$$min_{\beta_m, \gamma_m} \ \sum_{i=1}^{N}L(y_i, \sum_{m=1}^{M}\beta_mb(x_i;\gamma_m))$$

这个优化问题可以使用前向分布算法求解，其思想是从前到后，每一步只学习一个基函数及其系数，逐步逼近优化目标函数。具体说，就是每步优化损失函数：
$$min_{\beta, \gamma} \ \sum_{i=1}^{N}L(y_i, \beta b(x_i;\gamma))$$



![forward algorithm](pic/forward.png)

当损失函数$L(y, f(x)) = exp(-yf(x))$时，前向分布算法等价于Adaboost算法。

#### 2.1.3.4Boosting Tree（提升树）

以决策树为基函数的提升方法称为提升树，决策树既可以分类也可以回归，对分类问题决策树是二叉分类树，对回归问题决策树是二叉回归树。提升树模型可以表示为决策树的加法模型：
$$f_M(x) = \sum_{m=1}^{M}T(x;\Theta_m)$$

T表示决策树，$\Theta$为决策树参数，M为树的个数。
初始提升树可是设为$f_0(x)=0$。

跟上面相比，每个基函数系数直接写入到决策树中，第m步的模型是：
$$f_m(x) = f_{m-1}(x)+T(x;\Theta_m)$$
每一步确定决策树参数$\Theta$:
$$\hat \Theta_m = argmin_{\Theta_m} \sum_{i=1}^{N}L(y_i, f_{m-1}(x_i)+T(x_i;\Theta_m))$$

对于回归问题，采用平方误差损失函数时，
$$L(y, f(x))=(y-f(x))^2$$

$$L(y, f(x))=(y - f_{m-1}(x)-T(x;\Theta_m))^2$$

令：
$$r=y-f_{m-1}(x)$$

也就是拟合数据的残差。所以对回归问题的提升树算法来说，每次求解的决策树就是要拟合当前模型的残差。

整个思路可以这么考虑：
- 对于给定的训练数据，首先构造一棵回归树拟合，通常情况拟合不完全，还会有残差
- 对于剩下的残差，再继续构造回归另一棵回归树拟合，可以看作是对第一颗回归树的补充
- 如果没有达到要求，继续利用残差构造回归树，每一轮迭代是为了减小上一步模型的残差，这样残差就会越来越小
- 最终输出回归树的线性和

有点像数学中的泰勒公式，如果三次函数不能逼近原始函数的话可以加上四次项、五次项。

#### 2.1.3.5Gradient Boosting （梯度提升）

上面的提升树使用了平方误差作为损失函数，还可以使用其他损失函数比如绝对值、对数等。对于一般的函数，每一步优化并不容易，有学者提出了梯度提升算法，利用最速下降的方法。关键是利用损失函数的梯度在当前模型的值作为提升树算法中残差的近似值，然后利用近似的残差来拟合一个回归树（《统计机器学习》）。
![gradientBoosting](pic/gradientboosting.png)

近两年比较流行的XGBOOST工具包的基础也是梯度提升，算法流程和并行计算上做了优化，在kaggle竞赛中除了深度学习之外也取得了一席之地。

## 2.2通过处理数据特征构建差异性分类器
对训练数据抽取不同的输入特征子集分别进行训练，从而构建具有差异性的分类器。一般采用随机子空间，少量余留法（抽取最重要的一些特征），遗传算法等。

## 2.3对分类器的处理构建差异性分类器
指的就是通过改变一个算法的参数来生成有差异性的同质分类器，比如改变神经网络的网络拓扑结构就可以构建出不同的分类器，两层神经网络、三层神经网络等。

# 3.举例应用

In [3]:
#读取数据
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
print('Training Image Shape:', mnist.train.images.shape)
print('Training Label Shape:', mnist.train.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Training Image Shape: (55000, 784)
Training Label Shape: (55000,)


In [4]:
#基础决策树
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=11)
#训练模型
dt_model.fit(mnist.train.images, mnist.train.labels)
test_data, test_label = mnist.test.images, mnist.test.labels
preds = dt_model.predict(test_data)
print('Classifier Name:', "Decision Tree")
accuracy = accuracy_score(preds, test_label)
print('Accuracy: ', accuracy)

Classifier Name: Decision Tree
Accuracy:  0.873


单个决策树的预测效果比较差。

In [23]:
bg_model = BaggingClassifier(base_estimator=None, n_estimators=10, 
                             max_features=0.5, bootstrap=True, 
                             n_jobs=-1, random_state=11)
rf_model = RandomForestClassifier(n_estimators=10, max_features=0.5, 
                                  bootstrap=True, random_state=11)
ad_model = AdaBoostClassifier(n_estimators=50, learning_rate=0.1, random_state=11)
classifiers = [bg_model, rf_model, ad_model]
names = ['Bagging', 'Random Forest', 'Adaboost']

#遍历三个不同的分类
for name, classifier in zip(names, classifiers):
    classifier.fit(mnist.train.images, mnist.train.labels)
    preds = classifier.predict(mnist.train.images)
    print('*' * 20)
    print('Classifier Name:', name)
    accuracy = accuracy_score(preds, mnist.train.labels)
    print('Training Accuracy: ', round(accuracy, 4))
    preds = classifier.predict(test_data)
    accuracy = accuracy_score(preds, test_label)
    print('Testing Accuracy: ', round(accuracy, 4))

********************
Classifier Name: Bagging
Training Accuracy:  0.9991
Testing Accuracy:  0.9475
********************
Classifier Name: Random Forest
Training Accuracy:  0.9991
Testing Accuracy:  0.9482
********************
Classifier Name: Adaboost
Training Accuracy:  0.7234
Testing Accuracy:  0.73


In [27]:
ad_model = AdaBoostClassifier(n_estimators=200, learning_rate=0.1, random_state=11)
ad_model.fit(mnist.train.images, mnist.train.labels)
preds = ad_model.predict(test_data)
print('Classifier Name:', "Decision Tree")
accuracy = accuracy_score(preds, test_label)
print('Accuracy: ', accuracy)

Classifier Name: Decision Tree
Accuracy:  0.805


In [18]:
bg_model

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=11,
         verbose=0, warm_start=False)

Bagging和随机森林提高了预测准确度，Adaboost有些过拟合。

# 4.参考文献

1.李航，《统计机器学习》。

2.集成学习，http://www.cnblogs.com/GuoJiaSheng/p/4033584.html

3.集成学习，http://blog.csdn.net/jlei_apple/article/details/8168856

4.提升树方法， http://blog.csdn.net/dark_scope/article/details/24863289

5.Introduction to Statistical Learning

6.The Elements of Statistical Learning